In [9]:
# Setup: environment and versions
import os, sys, platform
print({
    "python": sys.executable,
    "version": sys.version.split()[0],
    "platform": platform.platform(),
})

# Ensure we are using the requested venv
assert sys.executable == "/Users/vuhung/Desktop/LangGraph-Adventures/02.Agent/.venv/bin/python", "Please run with the specified virtualenv python"


{'python': '/Users/vuhung/Desktop/LangGraph-Adventures/02.Agent/.venv/bin/python', 'version': '3.13.7', 'platform': 'macOS-15.6-arm64-arm-64bit-Mach-O'}


In [10]:
# Load environment variables
from dotenv import load_dotenv
from pathlib import Path

load_dotenv(Path("/Users/vuhung/Desktop/LangGraph-Adventures/02.Agent/.env.local"))
import os

for key in ["LANGSMITH_TRACING","LANGSMITH_API_KEY","TAVILY_API_KEY","GOOGLE_API_KEY","GEMINI_API_KEY"]:
    assert os.getenv(key), f"Missing env var: {key}"


In [11]:
# Build a minimal LangGraph agent using Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

class AgentState(TypedDict):
    question: str
    answer: str

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)

# Define a node that uses the LLM to answer
async def answer_node(state: AgentState) -> AgentState:
    resp = await llm.ainvoke(state["question"])  # returns AIMessage
    return {"answer": resp.content, "question": state["question"]}

# Wire graph
builder = StateGraph(AgentState)
builder.add_node("answer", answer_node)
builder.add_edge(START, "answer")
builder.add_edge("answer", END)
app = builder.compile()

# Run the agent
result = await app.ainvoke({"question": "Why sky is blue?"})
print(result["answer"])


The sky is blue because of how sunlight interacts with Earth's atmosphere, a phenomenon primarily explained by **Rayleigh scattering**.

Here's a breakdown:

1.  **Sunlight is White Light:** Sunlight, which appears white to our eyes, is actually made up of all the colors of the rainbow (the visible spectrum: red, orange, yellow, green, blue, indigo, violet). Each color has a different **wavelength** – red has the longest wavelength, and violet/blue have the shortest.

2.  **Earth's Atmosphere:** Our atmosphere is composed of tiny gas molecules, mainly nitrogen (about 78%) and oxygen (about 21%), along with other trace gases and some dust/water vapor.

3.  **Scattering of Light:** When sunlight enters the atmosphere, these tiny gas molecules scatter the light in all directions. The amount of scattering depends on the wavelength of the light and the size of the particles it hits.

4.  **Rayleigh Scattering:**
    *   The gas molecules in our atmosphere are much smaller than the wavelengt